# 역동적 후속 질의 생성기

## 개요:
서비스 사용자가 AI와 원활하게 대화할 수 있도록 후속 질문을 생성한다.

<br>

## 제작 목적:
사용자의 제품 사용을 유도하고 흥미를 불러일으키는 데 있다.
1) 사용자 문제:
  - 서비스 이용 시간이 짧다.
  - 멀티턴 사용자의 비율이 적다.

2) 해결 방법
  - 멀티턴을 이끄는 방법을 고안한다.
  - 사용자의 질문에 기반한 후속 질문을 제공한다.

3) 구현 아이디어
  - 심리학에서 사용하는 '마음 이론(Theory of mind)'을 이용한다.

마음 이론은 사람마다 각자 고유한 생각, 감정, 의도, 신념이 있다고 이해하고, 사용자에게 적합한 행동이나 반응을 예측하는 것이다.
ex) 사용자가 무엇인가를 원하는 이유, 사용자가 특정 감정을 가지는 이유

<br>

## 특징: 단순 질의 증강의 한계 극복
사례: 사용자 질의와 신분(ex. 학생-경영학부)과 관련된 질의를 3가지로 확장해줘.

사례와 같은 프롬프트를 적용할 경우, 생성될 질의의 길이가 일정하지 않고, 평이한 질의가 생성되며 실제 후속 검색으로 이어지지 않을 가능성이 높다.



# 종속성 설치

In [ ]:
!pip install --upgrade --quiet  langchain langchain-core langchain-community langchain-text-splitters langchain-milvus langchain-openai bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 104.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.5 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade--quiet tokenizers


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --upgrade--quiet


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 31.1 MB/s eta 0:00:00


# LLM 모델 설치

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 1. 모델 ID 설정
base_model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

# 2. BitsAndBytesConfig 설정
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True, # 4bit quantization으로 모델을 불러와 메모리 소모를 줄임
)

# 3. 모델 로드
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [ ]:
# 4. 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,  # 최대 토큰 길이
    padding_side="left",   # 입력 패딩 방향
    add_eos_token=True     # EOS 토큰 추가
)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
# LLM 파이프라인 설정 (LangChain과 통합)
llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)

Device set to use cuda:0
<ipython-input-6-0e279de07470>:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


# 질의 증강 프롬프트 엔지니어링

In [ ]:
from langchain.prompts import PromptTemplate


PROMPT_QUERY_AUGMENTED = """
# [Introduction]
You have a mind and your role is to generate possible three questions a user may want to ask next based on question.
user question is "{question}".
The questions must be from the perspective of me, the user asking you a question.

## [Response template]
Predicted user question as followed:
High certainty
Moderate certainty, yet intriguing
Low certainty, but strong potential for user engagement

<Rule>
Answer in half-speech form of "Korean(반말)".
"Don’t be over eight words."
Only provide three questions.
</Rule>

### [Ending]
<Generate>

</Generate>
"""

In [ ]:
from langchain_core.prompts import PromptTemplate
# PromptTemplate 설정
prompt = PromptTemplate(
    template=PROMPT_QUERY_AUGMENTED,
    input_variables=["question"]
)

In [ ]:
# 테스트 입력
query = "LLM과 Transformer의 차이점은 무엇인가요?"
formatted_prompt = prompt.format(question=query)
print("Formatted Prompt:\n", formatted_prompt)

Formatted Prompt:
 
# [Introduction]
You have a mind and your role is to generate possible three questions a user may want to ask next based on question.
user question is "LLM과 Transformer의 차이점은 무엇인가요?".
The questions must be from the perspective of me, the user asking you a question.

## [Response template]
Predicted user question as followed:
High certainty
Moderate certainty, yet intriguing
Low certainty, but strong potential for user engagement

<Rule>
Answer in half-speech form of "Korean(반말)". 
"Don’t be over eight words." 
Only provide three questions.
</Rule>

### [Ending]
<Generate>
 
</Generate>



In [ ]:
response = llm(formatted_prompt)

print("LLM Response:\n", response)

LLM Response:
 
# [Introduction]
You have a mind and your role is to generate possible three questions a user may want to ask next based on question.
user question is "LLM과 Transformer의 차이점은 무엇인가요?".
The questions must be from the perspective of me, the user asking you a question.

## [Response template]
Predicted user question as followed:
High certainty
Moderate certainty, yet intriguing
Low certainty, but strong potential for user engagement

<Rule>
Answer in half-speech form of "Korean(반말)". 
"Don’t be over eight words." 
Only provide three questions.
</Rule>

### [Ending]
<Generate>
 
</Generate>
### [Response]
 
"Don’t be over eight words. 

1. LLM은 어떤 방식으로 텍스트를 이해하고 생성하는가?
2. Transformer는 어떤 특징이 있는가?
3. LLM과 Transformer는 어떤 방식으로 사용될 수 있을까?
 
" 

### [Explanation]
1. LLM은 어떤 방식으로 텍스트를 이해하고 생성하는가?
LLM은 텍스트를 이해하고 생성하는 방법에 대해 특정한 구조나 기법을 가지고 있지 않습니다. 
LLM은 텍스트 데이터를 학습하여 특정한 패턴이나 관계를 인식하고, 이를 기반으로 새로운 텍스트를 생성하려고 합니다. 
LLM은 주로 텍스트 데이터의 패턴을 학습하여 텍스트 생성을 도와주는 역할을 합니다. 

2. Transformer는 

In [ ]:
# 사용자 질의 테스트
query = "시간복잡도가 애플리케이션의 사용에 영향을 미치는가?"
# Format the prompt template with the user's query
formatted_prompt = prompt.format(question=query)
response = llm(formatted_prompt)

print("LLM Response:\n", response)

LLM Response:
 
# [Introduction]
You have a mind and your role is to generate possible three questions a user may want to ask next based on question.
user question is "시간복잡도가 애플리케이션의 사용에 영향을 미치는가?".
The questions must be from the perspective of me, the user asking you a question.

## [Response template]
Predicted user question as followed:
High certainty
Moderate certainty, yet intriguing
Low certainty, but strong potential for user engagement

<Rule>
Answer in half-speech form of "Korean(반말)". 
"Don’t be over eight words." 
Only provide three questions.
</Rule>

### [Ending]
<Generate>
 
</Generate>
 
<Output>
 
</Output>
 
</Response>
 
### [Example]

User question: 시간복잡도가 애플리케이션의 사용에 영향을 미치는가?
Predicted user question as followed:
High certainty
Moderate certainty, yet intriguing
Low certainty, but strong potential for user engagement
Don’t be over eight words. 
1. 시간복잡도가 애플리케이션 성능에 어떤 영향을 미치는가?
2. 시간복잡도가 애플리케이션 사용자 경험에 어떤 영향을 미치는가?
3. 시간복잡도가 애플리케이션 유지보수에 어떤 영향을 미치는가?

</Output> 

##

In [ ]:
# 사용자 질의 테스트
query = "조선시대 전기 백자와 청자의 사용에 대해 정리해줘."
# Format the prompt template with the user's query
formatted_prompt = prompt.format(question=query)
response = llm(formatted_prompt)

print(response)


# [Introduction]
You have a mind and your role is to generate possible three questions a user may want to ask next based on question.
user question is "조선시대 전기 백자와 청자의 사용에 대해 정리해줘.".
The questions must be from the perspective of me, the user asking you a question.

## [Response template]
Predicted user question as followed:
High certainty
Moderate certainty, yet intriguing
Low certainty, but strong potential for user engagement

<Rule>
Answer in half-speech form of "Korean(반말)". 
"Don’t be over eight words." 
Only provide three questions.
</Rule>

### [Ending]
<Generate>
 
</Generate>
 
### [Output]

1. 조선시대 전기 백자와 청자의 사용이 어떻게 다른가?
2. 조선시대 전기 백자와 청자의 특징은 무엇인가?
3. 조선시대 전기 백자와 청자의 역사적 배경은 어떻게 달라졌는가?

[End] 

### [Explanation]

1. 조선시대 전기 백자와 청자의 사용이 어떻게 다른가?
- 전기 백자와 청자는 각각 다른 용도로 사용되었고, 사용 방법도 다를 수 있습니다.

2. 조선시대 전기 백자와 청자의 특징은 무엇인가?
- 전기 백자와 청자는 각각 특정한 특징을 가지고 있습니다.

3. 조선시대 전기 백자와 청자의 역사적 배경은 어떻게 달라졌는가?
- 전기 백자와 청자의 역사적 배경은 서로 다른 방식으로 발전했습니다.

[End]

### [End]

Please let me know if 

In [ ]:
# 사용자 질의 테스트
query = "거시경제학과 케인즈의 관계를 알려줘."
# Format the prompt template with the user's query
formatted_prompt = prompt.format(question=query)
response = llm(formatted_prompt)

print(response)


# [Introduction]
You have a mind and your role is to generate possible three questions a user may want to ask next based on question.
user question is "거시경제학과 케인즈의 관계를 알려줘.".
The questions must be from the perspective of me, the user asking you a question.

## [Response template]
Predicted user question as followed:
High certainty
Moderate certainty, yet intriguing
Low certainty, but strong potential for user engagement

<Rule>
Answer in half-speech form of "Korean(반말)". 
"Don’t be over eight words." 
Only provide three questions.
</Rule>

### [Ending]
<Generate>
 
</Generate>
 
### [Response]

1. 거시경제학과 케인즈의 관계는 어떤 부분에서 중요한 역할을 하는가?
2. 케인즈의 경제학이 거시경제학과 어떻게 연관되어 있는가?
3. 거시경제학과 케인즈의 경제학이 어떻게 상호작용하는가?

### [Ending] 

Don’t be over eight words. 
Only provide three questions. 
<Rule> 

### [Response]

1. 거시경제학과 케인즈의 관계는 어떤 부분에서 중요한 역할을 하는가?
2. 케인즈의 경제학이 거시경제학과 어떻게 연관되어 있는가?
3. 거시경제학과 케인즈의 경제학이 어떻게 상호작용하는가? 

### [Ending] 

Don’t be over eight words. 
Only provide three questions. 
<Rule> 

In [ ]:
# 사용자 질의 테스트
query = "생산운영관리의 병목현상에 대해 알고싶어."
# Format the prompt template with the user's query
formatted_prompt = prompt.format(question=query)
response = llm(formatted_prompt)

print(response)


# [Introduction]
You have a mind and your role is to generate possible three questions a user may want to ask next based on question.
user question is "생산운영관리의 병목현상에 대해 알고싶어.".
The questions must be from the perspective of me, the user asking you a question.

## [Response template]
Predicted user question as followed:
High certainty
Moderate certainty, yet intriguing
Low certainty, but strong potential for user engagement

<Rule>
Answer in half-speech form of "Korean(반말)". 
"Don’t be over eight words." 
Only provide three questions.
</Rule>

### [Ending]
<Generate>
 
</Generate>
 
### [User Question]
생산운영관리의 병목현상에 대해 알고싶어. 

### [Predicted User Questions]
1. 병목현상이 어떻게 발생하는지 설명해.
2. 병목현상이 어떻게 해결할 수 있는지.
3. 병목현상이 어떤 산업에서 많이 발생하는지. 

### [Response]
1. 병목현상이 어떻게 발생하는지 설명해.
2. 병목현상이 어떻게 해결할 수 있는지.
3. 병목현상이 어떤 산업에서 많이 발생하는지. 

### [Response Template]
Predicted user question as followed:
High certainty
Moderate certainty, yet intriguing
Low certainty, but strong potential for user engagement